# Optional LLM Enrichment for Enhanced Metadata

This notebook demonstrates how to use a local LLM (Ollama) to enhance metadata extraction.

## Why LLM Enrichment?

LLMs can:
- **Decode abbreviations**: sst → sea surface temperature
- **Infer context**: Guess institution from project names
- **Suggest use cases**: Identify potential applications
- **Add domain knowledge**: Recognize scientific domain
- **Generate summaries**: Create human-readable descriptions

## Important Notes

- ✅ **Completely Optional** - System works fine without it
- ✅ **Privacy Preserving** - Uses local Ollama, no data sent externally
- ⚠️ **Slower** - Adds ~2-5 seconds per file
- ⚠️ **Requires Ollama** - Install separately

## Setup Ollama

First, install Ollama if you haven't already:

```bash
# Install Ollama
curl -fsSL https://ollama.com/install.sh | sh

# Pull a model (choose one)
ollama pull llama3.2:3b      # Fast, 2GB, good quality
ollama pull llama3.2:8b      # Slower, 4GB, better quality
ollama pull mistral:7b       # Alternative

# Test it
ollama run llama3.2:3b
```

In [ ]:
# Setup
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from llm_enricher import LLMEnricher, DataInspector
from metadata_extractors import MetadataExtractor
from search_engine import FAIRSearchEngine

## Example 1: Test Ollama Connection

In [ ]:
# Test connection to Ollama
try:
    enricher = LLMEnricher(model="llama3.2:3b")
    print("✓ Connected to Ollama")
except Exception as e:
    print(f"✗ Could not connect to Ollama: {e}")
    print("\nMake sure Ollama is running:")
    print("  ollama serve")

## Example 2: Enrich Basic Metadata

In [ ]:
# Extract basic metadata first
sample_file = Path("sample_data/ocean_temperature.nc")

if sample_file.exists():
    extractor = MetadataExtractor()
    metadata = extractor.extract(sample_file)
    
    print("Original Metadata:")
    print("=" * 60)
    print(f"Filename: {metadata.get('filename')}")
    print(f"Title: {metadata.get('title')}")
    print(f"Institution: {metadata.get('institution')}")
    print(f"Variables: {list(metadata.get('variables', {}).keys())}")
else:
    print("Sample file not found. Run notebook 00 first.")

In [ ]:
# Enrich with LLM
if sample_file.exists():
    print("\nEnriching metadata with LLM...")
    print("(This may take a few seconds)\n")
    
    enriched = enricher.enrich_metadata(metadata)
    
    if 'llm_enrichment' in enriched:
        llm_data = enriched['llm_enrichment']
        
        print("LLM-Enriched Metadata:")
        print("=" * 60)
        
        if 'domain' in llm_data:
            print(f"Domain: {llm_data['domain']}")
        
        if 'source_institution' in llm_data:
            print(f"Inferred Institution: {llm_data['source_institution']}")
        
        if 'variable_descriptions' in llm_data:
            print(f"\nVariable Descriptions:")
            for var, desc in llm_data['variable_descriptions'].items():
                print(f"  {var}: {desc}")
        
        if 'use_cases' in llm_data:
            print(f"\nPotential Use Cases:")
            for use_case in llm_data['use_cases']:
                print(f"  - {use_case}")
        
        if 'quality_notes' in llm_data:
            print(f"\nQuality Notes: {llm_data['quality_notes']}")
    else:
        print(f"Enrichment error: {enriched.get('llm_error')}")

## Example 3: Data Inspection Tools

Inspect actual data values to validate metadata.

In [ ]:
# Inspect variable statistics
if sample_file.exists():
    inspector = DataInspector()
    
    print("Variable Statistics:")
    print("=" * 60)
    
    stats = inspector.get_variable_statistics(
        str(sample_file), 
        'sea_surface_temperature'
    )
    
    if 'error' not in stats:
        print(f"Variable: sea_surface_temperature")
        print(f"  Min: {stats['min']:.2f}")
        print(f"  Max: {stats['max']:.2f}")
        print(f"  Mean: {stats['mean']:.2f}")
        print(f"  Std Dev: {stats['std']:.2f}")
        print(f"  Shape: {stats['shape']}")
        print(f"  Data Type: {stats['dtype']}")
    else:
        print(f"Error: {stats['error']}")

In [ ]:
# Check temporal coverage
if sample_file.exists():
    print("\nTemporal Coverage:")
    print("=" * 60)
    
    temporal = inspector.check_temporal_coverage(str(sample_file))
    
    if 'error' not in temporal:
        print(f"Number of timesteps: {temporal['num_timesteps']}")
        print(f"Time units: {temporal['units']}")
        
        if 'start_date' in temporal:
            print(f"Start: {temporal['start_date']}")
            print(f"End: {temporal['end_date']}")
    else:
        print(f"Error: {temporal['error']}")

In [ ]:
# Check spatial coverage
if sample_file.exists():
    print("\nSpatial Coverage:")
    print("=" * 60)
    
    spatial = inspector.check_spatial_coverage(str(sample_file))
    
    if 'error' not in spatial:
        print(f"Latitude: {spatial['lat_min']:.2f}° to {spatial['lat_max']:.2f}°")
        print(f"Longitude: {spatial['lon_min']:.2f}° to {spatial['lon_max']:.2f}°")
        
        if spatial['lat_resolution']:
            print(f"Lat resolution: {spatial['lat_resolution']:.2f}°")
        if spatial['lon_resolution']:
            print(f"Lon resolution: {spatial['lon_resolution']:.2f}°")
    else:
        print(f"Error: {spatial['error']}")

## Example 4: Enrich Search Results

Add LLM-generated summaries to search results.

In [ ]:
# Search first
try:
    engine = FAIRSearchEngine(load_existing=True)
    results = engine.search("ocean temperature", top_k=3)
    
    if results:
        print("Original Search Results:")
        print("=" * 60)
        for i, r in enumerate(results, 1):
            print(f"\n{i}. {Path(r['filepath']).name}")
            print(f"   Score: {r['similarity_score']:.3f}")
            print(f"   Title: {r.get('title', 'N/A')}")
        
        # Enrich with LLM summaries
        print("\n" + "=" * 60)
        print("Generating LLM Summaries...")
        print("=" * 60)
        
        enriched_results = enricher.enrich_search_results(results)
        
        print("\nEnriched Results:")
        print("=" * 60)
        for i, r in enumerate(enriched_results, 1):
            print(f"\n{i}. {Path(r['filepath']).name}")
            print(f"   Score: {r['similarity_score']:.3f}")
            
            if 'llm_summary' in r:
                print(f"   Summary: {r['llm_summary']}")
    else:
        print("No results found. Index some data first.")

except FileNotFoundError:
    print("No index found. Run indexing first.")

## Example 5: Compare With/Without Enrichment

In [ ]:
# Create minimal metadata file
import netCDF4
import numpy as np

minimal_file = Path("sample_data/test_minimal.nc")

with netCDF4.Dataset(minimal_file, 'w') as ds:
    # NO global attributes, only variable name
    ds.createDimension('x', 100)
    ds.createDimension('y', 100)
    ds.createDimension('t', 10)
    
    var = ds.createVariable('wspd', 'f4', ('t', 'y', 'x'))
    var[:] = np.random.randn(10, 100, 100) * 5 + 10

print(f"Created minimal file: {minimal_file}")

In [ ]:
# Extract without enrichment
extractor = MetadataExtractor()
basic_metadata = extractor.extract(minimal_file)

print("Without LLM Enrichment:")
print("=" * 60)
print(f"Title: {basic_metadata.get('title') or '(none)'}")
print(f"Institution: {basic_metadata.get('institution') or '(none)'}")
print(f"Variables: {list(basic_metadata.get('variables', {}).keys())}")
print(f"\nSearchable text length: {len(extractor.create_searchable_text(basic_metadata))} chars")

In [ ]:
# Enrich with LLM
enriched_metadata = enricher.enrich_metadata(basic_metadata)

print("\nWith LLM Enrichment:")
print("=" * 60)

if 'llm_enrichment' in enriched_metadata:
    llm_data = enriched_metadata['llm_enrichment']
    
    print(f"Inferred domain: {llm_data.get('domain', 'N/A')}")
    print(f"\nVariable interpretation:")
    for var, desc in llm_data.get('variable_descriptions', {}).items():
        print(f"  {var} → {desc}")
    
    print(f"\nPotential uses:")
    for use in llm_data.get('use_cases', []):
        print(f"  - {use}")
    
    # Create enriched searchable text
    enriched_text = extractor.create_searchable_text(basic_metadata)
    enriched_text += " " + str(llm_data)
    print(f"\nSearchable text length: {len(enriched_text)} chars (increased by {len(enriched_text) - len(extractor.create_searchable_text(basic_metadata))}")
else:
    print(f"Error: {enriched_metadata.get('llm_error')}")

## Performance Considerations

In [ ]:
import time

# Time basic extraction
start = time.time()
basic = extractor.extract(sample_file)
basic_time = time.time() - start

# Time with LLM enrichment
start = time.time()
enriched = enricher.enrich_metadata(basic)
llm_time = time.time() - start

print("Performance Comparison:")
print("=" * 60)
print(f"Basic extraction: {basic_time:.3f} seconds")
print(f"LLM enrichment: {llm_time:.3f} seconds")
print(f"Total with LLM: {basic_time + llm_time:.3f} seconds")
print(f"\nSpeed difference: {((basic_time + llm_time) / basic_time):.1f}x slower with LLM")

print("\n💡 Recommendation:")
if llm_time < 5:
    print("  LLM enrichment is reasonably fast for your use case")
else:
    print("  Consider using LLM enrichment selectively for important datasets")

## When to Use LLM Enrichment

### ✅ Good Use Cases
- Files with minimal or cryptic metadata
- Variable names that are abbreviations
- Data from unfamiliar sources
- High-value datasets worth extra processing
- Interactive exploration (real-time is fine)

### ⚠️ Consider Skipping
- Large batch processing (1000s of files)
- Well-documented CF-compliant files
- Time-sensitive indexing workflows
- When basic search already works well

## Best Practices

1. **Index without LLM first** - Get basic search working quickly
2. **Enrich selectively** - Use LLM for important datasets
3. **Cache results** - Store enriched metadata
4. **Use local models** - Keep data private
5. **Monitor quality** - Verify LLM outputs make sense

## Next Steps

- **Notebook 99**: See complete workflows with optional enrichment
- Try different Ollama models for quality/speed tradeoffs
- Integrate enrichment into your indexing pipeline selectively